<a href="https://colab.research.google.com/github/kairamilanifitria/NLP-Projects/blob/main/Project%202%20Text%20Summarization/Fix/3_Modelling_BERT_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [3]:
# Load the dataset
train_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/train_df.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/dev_df.csv')

# Basic overview
print(train_df.shape)
print(train_df.info())
print(train_df.isnull().sum())

print(dev_df.shape)
print(dev_df.info())
print(dev_df.isnull().sum())

(29842, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29842 entries, 0 to 29841
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  29842 non-null  int64 
 1   original_text               29842 non-null  object
 2   abstractive_summary         29842 non-null  object
 3   extractive_summary          29842 non-null  object
 4   original_text_length        29842 non-null  int64 
 5   abstractive_summary_length  29842 non-null  int64 
 6   extractive_summary_length   29842 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.6+ MB
None
Unnamed: 0                    0
original_text                 0
abstractive_summary           0
extractive_summary            0
original_text_length          0
abstractive_summary_length    0
extractive_summary_length     0
dtype: int64
(3000, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
D

In [4]:
columns_to_delete = ['Unnamed: 0', 'extractive_summary', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']

# Delete the specified columns from the DataFrame
train_df = train_df.drop(columns=columns_to_delete)
dev_df = dev_df.drop(columns=columns_to_delete)

In [5]:
train_df.iloc[1]

,1
original_text,liputan6 com jakarta karyawan pt angkasa pura ...
abstractive_summary,karyawan pt angkasa pura i berencana mogok ker...


In [7]:
train_df.head()

,original_text,abstractive_summary
0,liputan6 com liverpool sepertinya tidak ada au...,kembalinya wayne rooney ke goodison park disam...
1,liputan6 com jakarta karyawan pt angkasa pura ...,karyawan pt angkasa pura i berencana mogok ker...
2,liputan6 com jakarta peluh membasahi kening pu...,calon penumpang mesti bolak balik datang karen...
3,liputan6 com ambon peningkatan status darurat ...,situasi keamanan di ambon maluku berangsur pul...
4,liputan6 com jakarta lima nelayan indonesia ya...,lima nelayan indonesia yang masuk perairan mal...


In [6]:
dev_df.iloc[1]

,1
original_text,liputan6 com jakarta peruntungan ahmad taufik ...
abstractive_summary,presiden menyempatkan diri untuk bertemu denga...


In [8]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [9]:
# Load pre-trained BERT model for text summarization (extractive)
model_name = "cahya/bert2bert-indonesian-summarization"
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [10]:
# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [11]:
from transformers import TrainingArguments, Trainer

def preprocess_function_abstractive(examples):

    abstractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["abstractive_summary"]
    ]  # List comprehension for handling list summaries

    # Tokenize the text input
    model_inputs = tokenizer(
        examples["original_text"], max_length=512, truncation=True, padding="max_length"
    )

    # Tokenize the combined summary (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            abstractive_summary, max_length=256, truncation=True, padding="max_length"
        )

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
# Apply preprocessing to the train and dev datasets
tokenized_train_datasets = train_dataset.map(preprocess_function_abstractive, batched=True)
tokenized_dev_datasets = dev_dataset.map(preprocess_function_abstractive, batched=True)

Map:   0%|          | 0/29842 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    fp16=True,
    save_steps=500,           # Save the model every 500 steps
    eval_steps=500,           # Evaluate every 500 steps
    logging_dir="./logs",     # Directory for logs
    logging_steps=100,        # Log every 100 steps
)


# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,  # Use tokenized train dataset
    eval_dataset=tokenized_dev_datasets,     # Use tokenized dev dataset for validation
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [14]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,0.147100,0.268721
2,0.106400,0.276080
3,0.079900,0.283338


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Some

TrainOutput(global_step=11193, training_loss=0.1217243020681277, metrics={'train_runtime': 8207.8641, 'train_samples_per_second': 10.907, 'train_steps_per_second': 1.364, 'total_flos': 5.49204982848553e+16, 'train_loss': 0.1217243020681277, 'epoch': 3.0})

In [15]:
import os

# Define the path in Google Drive where you want to save the model
output_dir = '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/BERT:Abstractive'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Model saved to /content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/BERT:Abstractive


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/test_df.csv')

In [ ]:
columns_to_delete = ['Unnamed: 0', 'extractive_summary', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']
# Delete the specified columns from the DataFrame
test_df = test_df.drop(columns=columns_to_delete)
test_df.head()

In [ ]:
test_df.shape

In [ ]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_abstractive_summaries_for_df(df):
    model_path = "/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/BERT:Abstractive"
    summarizer =EncoderDecoderModel.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = BertTokenizer.from_pretrained(model_path)

    # Lists to store both types of summaries
    abstractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            abstractive_summaries.append(abstractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            abstractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'abstractive_summary': abstractive_summaries
    })

# Assuming test_df_subset is already defined
abstractive_summary_BERT_df = generate_abstractive_summaries_for_df(test_df)

abstractive_summary_BERT_df

In [ ]:
from google.colab import files

csv_file_path = 'abstractive_summary_BERT_df.csv'
abstractive_summary_BERT_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

# **perplexity** CALCULATION